In [ ]:
import os, re
os.chdir("/Users/Pace/Documents/Education/Universität Potsdam/Semester 4 - Summer 2024/Evaluating Chatp-Optimized Language Models/Project")
from backends import ModelSpec, Model, get_model_for, load_model_registry

### Game logic

In [1]:
class Game:
    """
    Class describing the behavior of the game of Ludo, used to prompt an LLM.
    """
    def __init__(self, llm: Model, system_prompt: str, instructions: str) -> None:
        self.llm: Model = llm

        # Prompt and response attributes
        self.system_prompt: str = system_prompt
        self.instructions: str = instructions
        self.last_response: list = []
        self.context: list = []
        self.reprompt: bool = False

        # Constructs board with n fields
        self.n_fields: int = 23 # TODO Determine generative method for fields
        cells: list = []
        for _ in range(self.n_fields):
            cells.append("□")
        self.board = " ".join(cells).rstrip()
        self.current_state: str = self.board

        self.rolls: list = [6, 5, 4, 5, 6, 6, 2, 3, 6, 5, 1, 2, 5, 3, 4] # TODO Determine generative method for rolls
        self.turn: int = 0

        # Token attributes
        self.tokens_inplay: dict = {"X": False, "Y": False}
        self.current_position: dict = {"X": 0, "Y": 0}
        self.six_count: int = 0

    def make_move(self) -> None:
        """
        When called, prompts the LLM according to the turn number, giving it the previous conversation 
        context, and requests its next move. The move is parsed, its validity is determined, at which point 
        either its move is reflected on the board or an error message is returned and the game ends.
        """
        # Prepares the appropriate message for the current turn
        match self.turn:
            case 0:
                print(f"TURN 0: {self.board}")
                message: str = f"{self.instructions}\nAre you ready to play?\nBeginning state: {self.board}"
                message += f"\nTurn number: {self.turn}, Roll: {self.rolls[self.turn]}. "
                message += "Where will you move your token? Let's think step by step."

            case _:
                message: str = f"Current state: {self.current_state}"
                message += f"\nNext turn number: {self.turn}, Roll: {self.rolls[self.turn]}. "
                message += "Where will you move your token? Let's think step by step."

        # Delivers the message, then retrieves the response
        self._add_message(message)
        _, _, response_text = self.llm.generate_response(self.context)

        # Parses and saves the response
        move: dict = self._parse_reply(response_text)
        self.last_response = response_text

        # If the move is valid, updates everything accordingly
        if self._check_move(move, self.rolls[self.turn]):
            self._add_message(response_text, role="assistant")
            for token in move.keys():
                self.tokens_inplay[token] = move[token] > 0
                self.current_position[token] = move[token]
            self._update_board(move)
            self.turn += 1
            print(f"TURN {self.turn}: {self.current_state}")

        # Otherwise, reports relevant information at the failing turn
        else:
            print(f"Fail at turn {self.turn}")
            print(f"Roll: {self.rolls[self.turn]}")
            print(f"Move: {move}")
            print(response_text)
            print(self.current_state)

    def _add_message(self, message: str, role: str = "user") -> None:
        """
        Adds a message to the conversation context. If the conversation has just begun and the context 
        is empty, we start by adding in the system prompt.

        Args:
            message (str): message to be added to the context
            role (str): indicates to the LLM which role the message belongs to (i.e., 'system', 
                        'user', or 'assistant')
        """
        if not self.context:
            self.context = [{"role": "system", "content": self.system_prompt}]
            
        self.context.append({"role": role, "content": message})

    def _parse_reply(self, reply: str) -> dict[str: int]:
        """
        Parses the LLM's response according to a pre-described format, given in the instructions.

        Args:
            reply (str): the LLM's response to be parsed

        Returns:
            dict[str: int]: contains the LLM's move, described by the name of the token 
                            and the move destination

        Raises:
            ValueError: raised if the reply is not given in the described format
        """
        matching_string: Match = re.search(r"MY MOVE: X -> (\d+) ; Y -> (\d+)", reply)

        if not matching_string:
            raise ValueError("Invalid response format")

        return {"X": int(matching_string.group(1)), "Y": int(matching_string.group(2))}

    def _check_token_moved(self, move: dict[str: int]) -> dict[str: bool]:
        """
        Determines if the LLM has decided to move the tokens (on a per-token basis).

        Args:
            move (dict[str: int]): contains the LLM's move, described by the name of the token
                                   and the move destination

        Returns:
            dict[str: bool]: contains the name of the tokens and a bool, True if the token has 
                             been moved, False otherwise
        """
        tokens_moved: dict = {}

        for token in move.keys():
            tokens_moved[token] = self.current_position[token] != move[token]

        return tokens_moved

    def _check_both_tokens_moved(self, move: dict[str: int]) -> bool:
        """
        Determines if the LLM has decided to move both of its tokens in one turn.

        Args:
            move (dict[str: int]): contains the LLM's move, described by the name of the token
                                   and the move destination

        Returns:
            bool: True if both tokens were moved, False otherwise
        """
        return True if all([value for value in _check_token_moved(move).values()]) else False

    def _find_selected(self, tokens_moved: dict[str: bool]) -> str:
        """
        If a token was moved during a turn, determines which token was moved.

        Args:
            tokens_moved (dict[str: bool]): contains the name of the tokens and a bool, True if 
                                            the token has been moved, False otherwise

        Returns:
            str: name of the token which was moved during the turn
        """
        for token in tokens_moved.keys():
            if tokens_moved[token]:
                selected: str = token

        return selected

    def _check_move(self, move: dict[str: int], roll: int) -> bool:
        """
        Determines if a given move is legal, according to the game's rules and the die roll 
        given during the turn.
        
        Args:
            move (dict[str: int]): contains the LLM's move, described by the name of the token 
                                   and the move destination
            roll (int): the die roll for the turn

        Returns:
            bool: True if a move is valid, False otherwise
        """
        if self._check_both_tokens_moved(move):
            print("MOVE ERROR: Both in-play tokens were moved simultaneously.")
            return False

        state: list = []
        selected: str = self._find_selected(self._check_token_moved(move))

        for token in move.keys():
            match [selected, self.tokens_inplay[token]]:
                
                # if the token was not moved and has not been played to the board
                case [False, False]: 
                    if roll != 6:
                        state.append(True)
                        continue

                    else:
                        print(f"MOVE ERROR: A 6 was rolled but the token {token} was not played to the board.")
                        return False
                
                # if the token was not moved but has been played to the board
                case [False, True]:
                    if roll + self.current_position[token] > self.n_fields:
                        state.append(True)
                        continue

                    else:
                        print(f"MOVE ERROR: The token {token} can be moved but was not.")
                        return False
                
                # if the selected token was moved
                case [True, _]: 
                    match [token == selected]:
                        
                        # if the selected token has been moved
                        case True: 
                            if roll == 6 and move[token] == 1:
                                state.append(True)
                                continue

                            elif self.current_position[token] + roll == move[token]:
                                state.append(True)
                                continue

                            else:
                                print(f"MOVE ERROR: The token {token} was not moved appropriately.")
                                return False

                        # if the selected token has not been moved
                        case False: 
                            state.append(True)
                            continue
                            
        if all(state):
            return True

    # TODO Build reprompt functionality
    def _reprompt(self) -> None:
        pass

    def _update_board(self, move: dict[str: int]) -> None:
        """
        Updates the current state of the board given a valid move.

        Args:
            move (dict[str: int]): contains the LLM's move, described by the name of the token 
                                   and the move destination
        """
        split_board: list = self.board.split()
        
        for token in move.keys():
            position: int = move[token]
            if self.tokens_inplay[token]:
                split_board[self.current_position[token] - 1] = "□"
                split_board[position - 1] = token

        self.current_state = " ".join(split_board).rstrip()

    def _reset(self) -> None:
        self.turn = 0
        self.context = []

SyntaxError: unterminated string literal (detected at line 34) (1455880178.py, line 34)

### Game Instantiation

In [ ]:
# Prepares the LLM
load_model_registry()
THIS_MODEL = dict(model_id="gpt-3.5-turbo-1106", backend="openai", model_name = "gpt-3.5-turbo-1106")
llm: Model = get_model_for(THIS_MODEL)
llm.set_gen_args(temperature=0.0, max_tokens=400)

# Loads the prompts
with open('games/ludo/resources/initial_prompts/simple_prompt_v1.txt', 'r') as f:
    system_prompt: str = f.read()
with open('games/ludo/resources/prompts_pace/multitoken_v1.txt', 'r') as f:
    instructions: str = f.read()

# Defines the game instance
instance: Game = Game(llm, system_prompt, instructions)

In [ ]:
for _ in range(10):
    instance.make_move()

In [ ]:
print(instance.last_response)

In [ ]:
import shutil
width, _ = shutil.get_terminal_size()

for turn_content in instance.context:
    print("=" * width + "\n")
    print(f"Role: {turn_content['role']}")
    print(turn_content["content"] + "\n\n")